In [1]:
import numpy as np
from numpy import array, exp, sqrt
import pandas as pd
from copy import deepcopy
from fox_toolbox.utils import rates, volatility
from neuron.network import Network
from itertools import product
from datetime import datetime, date
from mkt_data import mkt_data

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

np.set_printoptions(precision=5)

% load_ext autoreload
% autoreload 2

In [2]:
strikes = np.linspace(0, 0.2, 3)
forwards = np.linspace(0, 0.2, 3)
maturities = np.linspace(1,3,3)
rs = np.linspace(0, 0.1, 2)

In [3]:
pd.DataFrame(product(strikes, forwards, maturities, rs), columns=['Strike', 'Forward', 'Maturity', 'Rate']).head(10)

,Strike,Forward,Maturity,Rate
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.1
2,0.0,0.0,2.0,0.0
3,0.0,0.0,2.0,0.1
4,0.0,0.0,3.0,0.0
5,0.0,0.0,3.0,0.1
6,0.0,0.1,1.0,0.0
7,0.0,0.1,1.0,0.1
8,0.0,0.1,2.0,0.0
9,0.0,0.1,2.0,0.1


In [4]:
swo_df = mkt_data.get_swo_df()
# swo_df.drop(['DATE', 'CURRENCY'], axis=1, inplace=True)
swo_df = swo_df[swo_df['STRIKE'].notna()]
swo_df = swo_df[(swo_df.NVOL != 0)]

In [5]:
tenor_to_val = {'1M':  1. / 12, 
                '2M':  2. /.12, 
                '3M':  3. /.12, 
                '6M':  6. / 12, 
                '9M':  9. / 12,
                '1Y':  1.,
                '18M': 18. /12,
                '2Y':  2.,
                '3Y':  3.,
                '4Y':  4.,
                '5Y':  5.,
                '6Y':  6.,
                '7Y':  7.,
                '8Y':  8.,
                '9Y':  9.,
                '10Y': 10.,
                '11Y': 11.,
                '12Y': 12.,
                '15Y': 15.,
                '20Y': 20.,
                '25Y': 25.,}

swo_df['expiry'] = swo_df["EXPIRY"].map(tenor_to_val)
swo_df['tenor'] = swo_df["TENOR"].map(tenor_to_val)
swo_df['fwd'] = swo_df.STRIKE/100. - swo_df.MONEYNESS

In [6]:
df = swo_df[(swo_df.DATE == swo_df.DATE[1]) & (swo_df.CURRENCY == 'USD') & (swo_df.EXPIRY == '2Y') & (swo_df.TENOR == '2Y')]

In [7]:
df['Bachelier'] = df.apply(lambda row: volatility.BachelierPrice(row.fwd, row.STRIKE/100., row.NVOL * sqrt(row.expiry), w=np.sign(row.MONEYNESS)), axis=1)

c:\users\mkapchenko\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
dfnn = df.drop(['DATE', 'CURRENCY', 'EXPIRY', 'TENOR', 'NVOL', 'Bachelier', 'fwd'], axis=1)

In [13]:
X = dfnn.to_numpy()

In [14]:
y = df.Bachelier.to_numpy()
y = y.reshape(-1, 1)

In [15]:
dfnn

STRIKE  MONEYNESS  expiry  tenor
DATE       CURRENCY                                  
2018-01-03 USD        0.371    -0.0200     2.0    2.0
           USD        1.371    -0.0100     2.0    2.0
           USD        1.871    -0.0050     2.0    2.0
           USD        2.121    -0.0025     2.0    2.0
           USD        2.621     0.0025     2.0    2.0
           USD        2.871     0.0050     2.0    2.0
           USD        3.371     0.0100     2.0    2.0
           USD        4.371     0.0200     2.0    2.0
           USD        5.371     0.0300     2.0    2.0
           USD        6.371     0.0400     2.0    2.0

In [18]:
X.shape

(10, 4)